[![Notebook Tutorial](__code/__all/notebook_tutorial.png)](https://neutronimaging.pages.ornl.gov/en/tutorial/notebooks/water_intake_profile_calculator/)

<img src='__docs/__all/notebook_rules.png' />

# Select your IPTS 

In [1]:
from __code.ui_builder import UiBuilder
o_builder = UiBuilder(ui_name = 'ui_water_intake_profile.ui')
from __code.roi_selection_ui import Interface

from __code import system
from __code.water_intake_profile_calculator import WaterIntakeProfileCalculator, WaterIntakeProfileSelector

system.System.select_working_dir()
from __code.__all import custom_style
custom_style.style()

pyuic5 /Users/j35/git/IPTS/python_notebooks/ui/ui_water_intake_profile.ui -o /Users/j35/git/IPTS/python_notebooks/__code/ui_water_intake_profile.py


** Using Debugging Mode! **


VBox(children=(HBox(children=(Label(value='IPTS-'), Text(value='', layout=Layout(width='10%')), Label(value='DOES NOT EXIST!', layout=Layout(width='20%'), _dom_classes=('result_label',)))), Label(value='OR'), HBox(children=(Label(value='Select Folder', layout=Layout(width='20%')), Select(layout=Layout(height='300px'), options=('BraggEdge', 'Brianne', 'DOE_letters', 'FITSimages', 'IPTS-12534', 'IPTS-13639_resonance_shawn', 'IPTS-14832', 'IPTS-15177', 'IPTS-16212', 'IPTS-16259', 'IPTS-16475', 'IPTS-16893', 'IPTS-17099', 'IPTS-17685', 'IPTS-17981', 'IPTS-18580', 'IPTS-18806', 'IPTS-18813', 'IPTS-19037', 'IPTS-19213', 'IPTS-19558', 'IPTS-19621', 'IPTS-19626', 'IPTS-19799', 'IPTS-19921-Charles', 'IPTS-20209-TOPAZ', 'IPTS-Das-Saikat', 'IPTS_test_normalization', 'IPTS_testing_time_spectra', 'ImagesToRegisterForGian', 'Indu', 'Keita_code', 'Pour_ibeatles_video', 'ResonanceImaging', 'SENSITIVE', 'SarmaGUI', 'SphereCoordinates3D', 'TempData', 'gian', 'iBeatles_debugging', 'shared'), value='BraggEdge'))), Button(button_style='info', description='HELP', style=ButtonStyle())))

# Python Import 

In [2]:
%gui qt

# Select Images to Process

In [3]:
o_water = WaterIntakeProfileCalculator(working_dir=system.System.get_working_dir())
o_water.select_data()

Button(button_style='info', description='HELP', style=ButtonStyle())

VBox(children=(Label(value='Select Images ...', layout=Layout(width='250px')), VBox(children=(HBox(children=(HBox(children=(Text(value='/Volumes/my_book_thunderbolt_duo/IPTS/IPTS-Das-Saikat', description='Location: ', layout=Layout(width='100%'), placeholder=''), Button(description='Jump', layout=Layout(border='1px solid gray', margin='5px 10px', width='100px'), style=ButtonStyle())), layout=Layout(border='1px solid lightgrey', margin='5px 50px 5px 5px', padding='3px', width='100%')),)), HBox(children=(VBox(children=(HBox(children=(Label(value='Search:'), Text(value='', layout=Layout(width='50%'))), layout=Layout(justify_content='flex-end')), SelectMultiple(description='Select', layout=Layout(height='260px', width='99%'), options=(' .', ' ..', ' .DS_Store                 |     Mon Apr 16 16:35:54 2018', ' only_data_of_interest     |     Directory', ' renamed_water200          |     Directory'), value=())), layout=Layout(width='80%')), VBox(children=(Button(description='Change directory', layout=Layout(border='1px solid gray', margin='5px 40px'), style=ButtonStyle()), Button(description='Select', layout=Layout(border='1px solid blue', margin='auto 40px 5px'), style=ButtonStyle())))), layout=Layout(border='1px solid lightgrey', margin='5px', padding='10px')))), HTML(value='')))

HBox(children=(Label(value='Retrieving Time Stamp', layout=Layout(width='20%')), IntProgress(value=0, layout=Layout(width='50%'), max=10)))

HBox(children=(Label(value='Loading sample', layout=Layout(width='20%')), IntProgress(value=0, max=10), Label(value='Time remaining:', layout=Layout(width='10%')), Label(value=' >> calculating << ')))

# Select Profile Region 

In [1]:
o_gui = WaterIntakeProfileSelector(dict_data=o_water.dict_files)
o_gui.show()

NameError: name 'WaterIntakeProfileSelector' is not defined

%DEBUGGING

In [1]:
from __code import system
from __code.water_intake_profile_calculator import WaterIntakeProfileCalculator, WaterIntakeProfileSelector

In [2]:
%gui qt

In [3]:
list_files = ['/Volumes/my_book_thunderbolt_duo/IPTS/IPTS-Das-Saikat/only_data_of_interest/image_00544.tif',
              '/Volumes/my_book_thunderbolt_duo/IPTS/IPTS-Das-Saikat/only_data_of_interest/image_00545.tif',
              '/Volumes/my_book_thunderbolt_duo/IPTS/IPTS-Das-Saikat/only_data_of_interest/image_00546.tif',
              '/Volumes/my_book_thunderbolt_duo/IPTS/IPTS-Das-Saikat/only_data_of_interest/image_00547.tif',
              '/Volumes/my_book_thunderbolt_duo/IPTS/IPTS-Das-Saikat/only_data_of_interest/image_00548.tif',
             ]

o_water = WaterIntakeProfileCalculator()
o_water.load_and_plot(list_files)
o_gui = WaterIntakeProfileSelector(dict_data = o_water.dict_files)
o_gui.show()

HBox(children=(Label(value='Retrieving Time Stamp', layout=Layout(width='20%')), IntProgress(value=0, layout=Layout(width='50%'), max=5)))

HBox(children=(Label(value='Loading sample', layout=Layout(width='20%')), IntProgress(value=0, max=5), Label(value='Time remaining:', layout=Layout(width='10%')), Label(value=' >> calculating << ')))

[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169]


Exception: X and Y arrays must be the same shape--got (170,) and (171,).

In [6]:
import numpy as np

In [5]:
nbr_points = 20
data = np.arange(0,nbr_points,1)
print("data: {}".format(data))
print("len(data): {}".format(len(data)))

data: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]
len(data): 20


In [14]:
bin_array = []
bin_size = 2
step = bin_size
index = 0
while (index < len(data)-1):
    if np.mod(index,bin_size) != 0:
        pass
    else: #edge of over bin
        bin_array.append(index)
        
    index += 1
    
bin_array   

[0, 2, 4, 6, 8, 10, 12, 14, 16, 18]

In [49]:
bin_size = 2
step = np.int((nbr_points)/bin_size)
bins = np.linspace(0, nbr_points, step)
bins

array([ 0.        ,  2.22222222,  4.44444444,  6.66666667,  8.88888889,
       11.11111111, 13.33333333, 15.55555556, 17.77777778, 20.        ])

In [50]:
digitized = np.digitize(data, bins)
digitized

array([1, 1, 1, 2, 2, 3, 3, 4, 4, 5, 5, 5, 6, 6, 7, 7, 8, 8, 9, 9])

In [51]:
bin_means = [data[digitized == i].mean() for i in range(1, len(bins))]

In [52]:
bin_means

[1.0, 3.5, 5.5, 7.5, 10.0, 12.5, 14.5, 16.5, 18.5]